## Import

In [234]:
import math
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from surprise import Reader, Dataset, SVD
from surprise import KNNBasic
from surprise import accuracy
from surprise import accuracy

from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

sns.set_style("darkgrid")

### Importare dataset

In [235]:
df1 = pd.read_csv('./combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape:',df1.shape)

# Mi genera un nuovo indice da 1
df1 = df1.reset_index()
print('Dataset 1 shape without rating null:',df1.shape)

print('-Dataset examples-')
print(df1[:5])


Dataset 1 shape: (24058263, 2)
Dataset 1 shape without rating null: (24058263, 3)
-Dataset examples-
   index  Cust_Id  Rating
0      0       1:     NaN
1      1  1488844     3.0
2      2   822109     5.0
3      3   885013     4.0
4      4    30878     4.0


### Prendo i primi x , così che il calcolo del film non sia lunghissimo m genero tante colonne di film

In [236]:
df1 = df1.head(2000000)

### df_nan è un DataFrame che indica True per le righe in cui la colonna 'Rating' è nulla e False altrimenti. Quindi, ogni volta che c'è una transizione da False a True nella colonna 'Rating', rappresenta l'inizio di un nuovo "film"

In [237]:
df_nan = pd.DataFrame(pd.isnull(df1.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df1) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

#print('Movie numpy:',movie_np)
#print('Length:',len(movie_np))

df = df1[pd.notnull(df1['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df)


-Dataset examples-
           index  Cust_Id  Rating  Movie_Id
1              1  1488844     3.0         1
2              2   822109     5.0         1
3              3   885013     4.0         1
4              4    30878     4.0         1
5              5   823519     3.0         1
...          ...      ...     ...       ...
1999995  1999995   719099     3.0       361
1999996  1999996  2600755     5.0       361
1999997  1999997  1143715     4.0       361
1999998  1999998   555073     3.0       361
1999999  1999999   246973     5.0       361

[1999639 rows x 4 columns]


/tmp/ipykernel_4104/110950279.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Movie_Id'] = movie_np.astype(int)
/tmp/ipykernel_4104/110950279.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cust_Id'] = df['Cust_Id'].astype(int)


### Creo matrice di rating

In [238]:
ratings_matrix = df.pivot(index='Cust_Id', columns='Movie_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix = ratings_matrix.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix)

Movie_Id  1    2    3    4    5    6    7    8    9    10   ...  352  353  \
Cust_Id                                                     ...             
6         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
7         0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  ...  0.0  0.0   
10        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
25        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
33        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2649401   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2649404   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2649409   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2649426   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2649429   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

### Prendo i primi y così che anche lo user based ha tanti film (un buon profilo ricco) ma riduzo il numero di utenti per non renderlo infinito

In [239]:
ratings_matrix = ratings_matrix.head(2000)

### rimuovo le righe nulle

In [240]:
print(ratings_matrix.shape)

(2000, 361)


In [241]:
ratings_matrix_support = pd.DataFrame()
for i in ratings_matrix.iterrows():
    val = [not(j) for j in i[1]]
    #print(i[1])
    #print(all(val))
    if not(all(val)):
        ratings_matrix_support = ratings_matrix_support.append(i[1])

/tmp/ipykernel_4104/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_4104/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_4104/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_4104/3795530229.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_4104/3795530229.py:7: FutureWarning: The frame.append

KeyboardInterrupt: 

In [143]:
#ratings_matrix = ratings_matrix_support.copy()
print(ratings_matrix_support.shape)

(1000, 225)


# Creo Test set

### prendo il rating da colore che ne hanno almeno due e lo metto a 0

In [242]:
y_test = {}
count = 0
for i in ratings_matrix.iterrows():
    if count < 20:
        almenoUno = 0
        for j in range(1,len(i[1])):
            if i[1][j] != 0:
                if almenoUno:
                    y_test[(i[0],j)] = i[1][j]
                    i[1][j] = 0
                    count +=1
                    break
                else:
                    almenoUno= True
            

In [145]:
y_test

{(6, 157): 3.0,
 (7, 28): 4.0,
 (10, 191): 4.0,
 (42, 191): 3.0,
 (59, 180): 2.0,
 (79, 84): 3.0,
 (87, 111): 2.0,
 (97, 167): 4.0,
 (134, 55): 5.0,
 (183, 197): 4.0,
 (188, 108): 3.0,
 (192, 213): 4.0,
 (195, 175): 5.0,
 (199, 111): 4.0,
 (201, 30): 5.0,
 (261, 191): 2.0,
 (265, 30): 5.0,
 (266, 175): 5.0,
 (268, 58): 5.0,
 (283, 148): 1.0}

## Usare user based NN

#### Calcolo rating medi degli utenti

In [243]:
rating_medi = {}
# calcolo rating medio senza considerare gli 0
#for i in ratings_matrix.iterrows():
for i in ratings_matrix.iterrows():
        #print(i[0])
        count = 0
        countn = 0
        for j in i[1].values:
            if j != 0:
                  count += j
                  countn += 1
        if countn != 0:
                rating_medi[i[0]] = count/countn
        else:
                rating_medi[i[0]] = 0

print(rating_medi)

{6: 3.4444444444444446, 7: 4.6, 10: 3.0, 25: 3.0, 33: 4.0, 42: 3.6666666666666665, 59: 3.8, 79: 3.272727272727273, 87: 3.7142857142857144, 94: 3.6666666666666665, 97: 3.0, 116: 4.0, 126: 5.0, 131: 2.25, 134: 4.785714285714286, 142: 4.0, 158: 3.0, 164: 5.0, 168: 4.0, 169: 3.5, 178: 3.0, 183: 4.0, 188: 3.25, 192: 3.6666666666666665, 195: 3.4, 199: 4.076923076923077, 201: 3.5555555555555554, 242: 3.3333333333333335, 247: 3.0, 250: 5.0, 261: 3.0, 265: 3.4545454545454546, 266: 4.25, 268: 4.75, 283: 3.375, 291: 3.75, 296: 3.2, 298: 4.0, 301: 4.0, 302: 3.5, 304: 4.0, 305: 3.8333333333333335, 307: 3.75, 308: 3.0, 310: 4.0, 314: 5.0, 330: 3.4, 333: 4.5, 352: 5.0, 358: 2.0, 363: 5.0, 368: 2.0, 369: 4.0, 383: 3.375, 384: 4.0, 385: 4.0, 416: 5.0, 424: 3.6666666666666665, 437: 2.5555555555555554, 439: 2.5, 440: 4.0, 442: 4.0, 462: 2.8181818181818183, 470: 4.0, 471: 3.0, 477: 3.5714285714285716, 478: 3.25, 481: 4.357142857142857, 485: 3.0, 490: 5.0, 491: 3.5, 492: 2.5, 495: 4.5, 508: 3.8, 527: 3.9, 

### fun di similarità

In [244]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent

def user_similarity(ratings,user_target,id_user_target):
    similarità = {}
    # scorro UTENTI
    for i in ratings.iterrows():
        # se l'UTENTE è diverso dall'utente target
        if i[0] != id_user_target:
            # calcolo similarità
            
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                #if user_target[j] != 0:
                numeratore += (i[1][j]-rating_medi[i[0]])*(user_target[j]-rating_medi[id_user_target])
                
                denominatoreP1 += (i[1][j]-rating_medi[i[0]])**2
                
                denominatoreP2 +=(user_target[j]-rating_medi[id_user_target])**2
            
            sim = 0
            if denominatoreP1 !=0 and denominatoreP2 != 0:
                sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            #print(i[0],id_user_target)
            similarità[(i[0],id_user_target)] = sim
    return similarità

### pre-processing sulla similarità

In [245]:
similarita_coppie = {}
for i,_ in y_test:
    # Calcolare la similarità tra utenti
    similarita_coppie.update(user_similarity(ratings_matrix,ratings_matrix.loc[i],i))

In [246]:
similarita_coppie

{(7, 6): 0.9784268458316764,
 (10, 6): 0.989652374598463,
 (25, 6): 0.9852835726049816,
 (33, 6): 0.987728444993573,
 (42, 6): 0.9876828632437893,
 (59, 6): 0.982313344624046,
 (79, 6): 0.975989229304122,
 (87, 6): 0.9827662254653449,
 (94, 6): 0.9854905771040758,
 (97, 6): 0.9801621338461332,
 (116, 6): 0.9873055009182188,
 (126, 6): 0.9852835726049829,
 (131, 6): 0.9873003841590142,
 (134, 6): 0.974235170412727,
 (142, 6): 0.9852835726049751,
 (158, 6): 0.9852835726049816,
 (164, 6): 0.991188722406431,
 (168, 6): 0.987728444993573,
 (169, 6): 0.9856532194005041,
 (178, 6): 0.9877284449935796,
 (183, 6): 0.9817984228398277,
 (188, 6): 0.9804688418907611,
 (192, 6): 0.9823340871974068,
 (195, 6): 0.9832298680307207,
 (199, 6): 0.9820871358147586,
 (201, 6): 0.9779401196859187,
 (242, 6): 0.9823198028059236,
 (247, 6): 0.9836659344781656,
 (250, 6): 0.9838438970002539,
 (261, 6): 0.9878906795838912,
 (265, 6): 0.9764648355642256,
 (266, 6): 0.9895658177260606,
 (268, 6): 0.9829799321734

In [84]:
utenteSimileUno = ratings_matrix.loc[261]
utenteSimileDue = ratings_matrix.loc[2044720]
print(utenteSimileUno)
print(utenteSimileDue)
print(user_target)


Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    4.0
Name: 261, dtype: float64
Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    4.0
Name: 2044720, dtype: float64
Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    3.0
Name: 2442, dtype

In [263]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
def calcoloSimilaritaConSoglia(similarita):
    sim_soglia = {}
    for i in similarita:
        if similarita[i] > 0.90:
            sim_soglia[i] = similarita[i]
    return sim_soglia

#### fun Prediction

In [264]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_user_based(item,user_similarities,ratings_matrix,rating_medi,id_user_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    pred_ratings = 0
    # verifico che per quell'item il rating sia 0
    #if ratings_matrix.loc[id_user_target][item] == 0:
    for i in user_similarities:
        #if ratings_matrix.loc[i[0]].values[item] != 0:
        num += (user_similarities[i]* (ratings_matrix.loc[i[0]][item]-rating_medi[i[0]]))
        den += user_similarities[i]
    if den != 0:
        pred_ratings = rating_medi[id_user_target] + (num / den)
    else:
        pred_ratings = rating_medi[id_user_target]
    print(pred_ratings)
    return pred_ratings
    #else:
    #    return ratings_matrix.loc[id_user_target][item]

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_user_based(id_item_target, sim_soglia,ratings_matrix,rating_medi,id_user_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


### testing model

In [265]:
# Ottenere le previsioni per tutte le valutazioni
y_pred = {}

for i in y_test:
    print(f'{i} starting...')
    # vettore rating utente target
    print(f'user target processing...')
    user_target = ratings_matrix.loc[i[0]]
    
    print('sim_soglia processing...')
    # SIMILARITÀ SOPRA CERTA SOGLIA
    sim_soglia = calcoloSimilaritaConSoglia(similarita_coppie)
    # predico rating test
    print('predicting...')
    predicted_ratings_user_based = predict_user_based(i[1], sim_soglia, ratings_matrix,rating_medi,i[0])   
    if predicted_ratings_user_based < 0:
        predicted_ratings_user_based = 0
    # rating predetto
    print('Appending...')
    y_pred[i]=round(predicted_ratings_user_based,2)


(6, 157) starting...
user target processing...
sim_soglia processing...
predicting...
-0.2449805915135217
Appending...
(7, 28) starting...
user target processing...
sim_soglia processing...
predicting...
1.317132889295837
Appending...
(10, 191) starting...
user target processing...
sim_soglia processing...
predicting...
0.42546483802362545
Appending...
(42, 191) starting...
user target processing...
sim_soglia processing...
predicting...
1.092131504690292
Appending...
(59, 180) starting...
user target processing...
sim_soglia processing...
predicting...
0.127912659153389
Appending...
(79, 84) starting...
user target processing...
sim_soglia processing...
predicting...
-0.4002462748305824
Appending...
(87, 111) starting...
user target processing...
sim_soglia processing...
predicting...
0.2871984270439465
Appending...
(94, 257) starting...
user target processing...
sim_soglia processing...
predicting...
0.29026657071502804
Appending...
(97, 167) starting...
user target processing...
sim

In [266]:
print(f'y_pred:{y_pred}')
print(f'y_test:{y_test}')

y_pred:{(6, 157): 0, (7, 28): 1.32, (10, 191): 0.43, (42, 191): 1.09, (59, 180): 0.13, (79, 84): 0, (87, 111): 0.29, (94, 257): 0.29, (97, 167): 0, (116, 257): 0.62, (131, 241): 0, (134, 55): 1.11, (158, 334): 0, (164, 288): 1.32, (169, 241): 0.35, (183, 197): 1.14, (188, 108): 0, (192, 213): 0.0, (195, 175): 0.77, (199, 111): 0.65}
y_test:{(6, 157): 3.0, (7, 28): 4.0, (10, 191): 4.0, (42, 191): 3.0, (59, 180): 2.0, (79, 84): 3.0, (87, 111): 2.0, (94, 257): 2.0, (97, 167): 4.0, (116, 257): 4.0, (131, 241): 3.0, (134, 55): 5.0, (158, 334): 4.0, (164, 288): 3.0, (169, 241): 4.0, (183, 197): 4.0, (188, 108): 3.0, (192, 213): 4.0, (195, 175): 5.0, (199, 111): 4.0}


### MAE, NMAE, RMSE, Precision, Recall, F1

In [267]:
# i'm deleting the rate predicted for evaluationg sim_soglia
""" for i in range(len(y_true)):rating_medi[id_user_target]
    if y_true[i] == 0:
        y_pred[i] = 0 """

# Calcola le metriche aggiuntive

precision = precision_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
recall = recall_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
f1 = f1_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
#nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
#mae = accuracy.mae(y_pred)

def mae(true,pred):
    n = len(true)
    tot_val = 0
    for i in true:
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'MAE:{(tot_val/n)}')

def Nmae(true,pred):
    n = len(true)
    tot_val = 0
    max = 0
    min = 9999
    for i in true:
        if true[i] < min:
            min = true[i]
        if true[i] > max:
            max = true[i]
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'NMAE:{(tot_val/n)}')

def rmse(true,pred):
    n = len(true)
    tot_val = 0
    for i in true:
        val = (pred[i]-true[i])**2
        tot_val += val
    rapp = tot_val/n
    print(f'RMSE:{math.sqrt(rapp)}')

mae(y_test,y_pred)
Nmae(y_test,y_pred)
rmse(y_test,y_pred)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
#print(f'NMAE:',nmae)
#print(f'MAE:',mae)

MAE:3.0245
NMAE:3.0245
RMSE:3.1378089489323595
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


## Usare item based NN 

In [268]:
ratings_matrix_transpose = ratings_matrix.transpose()
#df.pivot(index='Movie_Id', columns='Cust_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix_transpose = ratings_matrix_transpose.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix_transpose)

Cust_Id   6      7      10     25     33     42     59     79     87     \
Movie_Id                                                                  
1           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
357         0.0    5.0    0.0    0.0    0.0    3.0    4.0    0.0    4.0   
358         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
359         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
360         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
361         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

Cust_Id   94     ...  15

### fun adjusted cosine similarity

In [269]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent
def item_similarity(ratings,item_target,id_item_target):
    similarita_transpose = {}
    # scorro UTENTI
    for i in ratings.iterrows():
        # i riga film con rating utenti
        # se l'ITEM è diverso dall'item target
        if i[0] != id_item_target:
            # calcolo similarità
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                #print(j)
                # j utente
                #if i[1][j] != 0 and item_target[j] != 0:
                numeratore += (i[1][j]-rating_medi[j])*(item_target[j]-rating_medi[j])
                denominatoreP1 += (i[1][j]-rating_medi[j])**2
                denominatoreP2 +=(item_target[j]-rating_medi[j])**2
            sim = 0
            if denominatoreP1 != 0 and denominatoreP2 != 0:
                sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            similarita_transpose[(i[0],id_item_target)] = sim

    return similarita_transpose
# Calcolare la similarità tra utenti
#item_similarity(ratings_matrix_transpose,item_target)
#print(similarita_transpose)

### Pre processing similarity

In [270]:
similarita_coppie_transpose = {}
for i,j in y_test:
    #print(i)
    # Calcolare la similarità tra utenti
    similarita_coppie_transpose.update(item_similarity(ratings_matrix_transpose,ratings_matrix_transpose.loc[j],j))

In [309]:
similarita_coppie_transpose

{(1, 157): 0.9974693794597435,
 (2, 157): 0.9991046957428655,
 (3, 157): 0.996077750984184,
 (4, 157): 0.9992452377485519,
 (5, 157): 0.9982258799391762,
 (6, 157): 0.9983662490844886,
 (7, 157): 0.9995439404537263,
 (8, 157): 0.9820349481710782,
 (9, 157): 0.9995439404537263,
 (10, 157): 0.9995439404537263,
 (11, 157): 0.9993858247675428,
 (12, 157): 0.998946742859342,
 (13, 157): 0.9995439404537263,
 (14, 157): 0.9991046684294942,
 (15, 157): 0.9988236374742407,
 (16, 157): 0.9978382015509598,
 (17, 157): 0.9929880036341767,
 (18, 157): 0.980645647500336,
 (19, 157): 0.9991924476476677,
 (20, 157): 0.9995439404537263,
 (21, 157): 0.9988235658857101,
 (22, 157): 0.9993153054054174,
 (23, 157): 0.9987704883266237,
 (24, 157): 0.9977323644800967,
 (25, 157): 0.9965014848470455,
 (26, 157): 0.9932698469865467,
 (27, 157): 0.9995263283420717,
 (28, 157): 0.9386706865334307,
 (29, 157): 0.9978211328274805,
 (30, 157): 0.7973189483727042,
 (31, 157): 0.9988235473718576,
 (32, 157): 0.994827

In [118]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
sim_soglia_transpose = {}
for i in similarita_transpose:
    if similarita_transpose[i] > 0.0:
        sim_soglia_transpose[i] = similarita_transpose[i]
print(sim_soglia_transpose)

{(2, 1): 0.045871176736922314}


### selecting item simili

In [310]:
# prendo solo gli item votati dall'utente target
def similaritaSoglia(similarita_transpose):
    sim_soglia_transpose = {}
    for i in similarita_transpose:
        if user_target[i[0]] > 0.99:
            sim_soglia_transpose[i] = similarita_transpose[i]
    return sim_soglia_transpose

### fun predict

In [311]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_item_based(user,sim_soglia_transpose,ratings_matrix_transpose,id_item_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    #scorro item simili a tutti i target
    for i in sim_soglia_transpose:
        # controllo che sia simili al target passato
        if i[1] == id_item_target:
            # se l'utente l'ha votato
            if ratings_matrix_transpose.loc[i[0]][user] != 0:
                num += (sim_soglia_transpose[i]* (ratings_matrix_transpose.loc[i[0]][user]))
                den += sim_soglia_transpose[i]
    pred_ratings = 0
    if den != 0:
        pred_ratings = (num / den)
    return pred_ratings
    

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_item_based(id_item_target, similarita_transpose,ratings_matrix_transpose,id_item_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


### Testing model

In [ ]:
scorro utenti target -> per ogni utente scorro i film simili a quello target -> calcoli

In [312]:
# Ottenere le previsioni per tutte le valutazioni
y_pred_transpose = {}

# scorro utenti target
for i in y_test:
    print(f'{i} starting...')
    # vettore rating utente target
    print(f'user target processing...')
    #item_target = ratings_matrix_transpose.loc[i[1]]
    
    print('sim_soglia processing...')
    # SIMILARITÀ SOPRA CERTA SOGLIA
    sim_soglia_transpose = similaritaSoglia(similarita_coppie_transpose)
    # predico rating test
    print('predicting...')
    predicted_ratings_item_based = predict_item_based(i[0], sim_soglia_transpose, ratings_matrix_transpose,i[1])   
    
    if predicted_ratings_item_based < 0:
        predicted_ratings_item_based = 0
    # rating predetto
    print('Appending...')
    y_pred_transpose[i]=round(predicted_ratings_item_based,2)


(6, 157) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(7, 28) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(10, 191) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(42, 191) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(59, 180) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(79, 84) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(87, 111) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(94, 257) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(97, 167) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(116, 257) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(13

In [313]:
print(f'y_pred:{y_pred_transpose}')
print(f'y_true:{y_test}')

y_pred:{(6, 157): 3.41, (7, 28): 4.56, (10, 191): 3.33, (42, 191): 4.0, (59, 180): 3.74, (79, 84): 3.23, (87, 111): 3.96, (94, 257): 4.48, (97, 167): 2.77, (116, 257): 4.0, (131, 241): 2.0, (134, 55): 4.84, (158, 334): 0, (164, 288): 5.0, (169, 241): 3.46, (183, 197): 3.96, (188, 108): 3.19, (192, 213): 4.0, (195, 175): 3.37, (199, 111): 4.07}
y_true:{(6, 157): 3.0, (7, 28): 4.0, (10, 191): 4.0, (42, 191): 3.0, (59, 180): 2.0, (79, 84): 3.0, (87, 111): 2.0, (94, 257): 2.0, (97, 167): 4.0, (116, 257): 4.0, (131, 241): 3.0, (134, 55): 5.0, (158, 334): 4.0, (164, 288): 3.0, (169, 241): 4.0, (183, 197): 4.0, (188, 108): 3.0, (192, 213): 4.0, (195, 175): 5.0, (199, 111): 4.0}


### MAE, NMAE, RMSE, Precision, Recall, F1

In [314]:
# i'm deleting the rate predicted for evaluationg sim_soglia
""" for i in range(len(y_true)):
    if y_true[i] == 0:
        y_pred[i] = 0 """


precision = precision_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
recall = recall_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
f1 = f1_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
#nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
#mae = accuracy.mae(y_pred)

def mae(true,pred):
    n = len(true)
    tot_val = 0
    for i in pred:
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'MAE:{1-(tot_val/n)}')

def Nmae(true,pred):
    n = len(true)
    tot_val = 0
    max = 0
    min = 9999
    for i in pred:
        if true[i] < min:
            min = true[i]
        if true[i] > max:
            max = true[i]
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'NMAE:{1-(tot_val/n)}')

def rmse(true,pred):
    n = len(true)
    tot_val = 0
    for i in pred:
        val = (pred[i]-true[i])**2
        tot_val += val
    rapp = tot_val/n
    print(f'RMSE:{1-math.sqrt(rapp)}')

mae(y_test,y_pred_transpose)
Nmae(y_test,y_pred_transpose)
rmse(y_test,y_pred_transpose)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
#print(f'NMAE:',nmae)
#print(f'MAE:',mae)

MAE:0.0044999999999999485
NMAE:0.0044999999999999485
RMSE:-0.42359931160421693
Precision: 0.4
Recall: 0.4
F1 Score: 0.4000000000000001


## Paradigma : User based vs item base

## Matrix factorization: SVD 

In [28]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:], reader)
#data.split(n_folds=3)
trainsetData, testsetData = train_test_split(data, test_size=0.2)

In [29]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'],verbose=True,return_train_measures=True,n_jobs=100)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0288  1.1406  1.0539  1.1039  1.1003  1.0855  0.0395  
MAE (testset)     0.8150  0.9319  0.8590  0.8994  0.8902  0.8791  0.0396  
RMSE (trainset)   0.7032  0.6807  0.7440  0.7037  0.6914  0.7046  0.0215  
MAE (trainset)    0.5630  0.5437  0.5911  0.5624  0.5548  0.5630  0.0157  
Fit time          0.02    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.0287584 , 1.14058036, 1.05386591, 1.10385707, 1.10030391]),
 'train_rmse': array([0.70317169, 0.68065717, 0.7440008 , 0.70374151, 0.69138451]),
 'test_mae': array([0.81496071, 0.93190821, 0.85901677, 0.89935797, 0.89016553]),
 'train_mae': array([0.56303226, 0.54372737, 0.59106961, 0.56243586, 0.55479555]),
 'fit_time': (0.01925039291381836,
  0.009661674499511719,
  0.010292768478393555,
  0.01019906997680664,
  0.010210752487182617),
 'test_time': (0.001659393310546875,
  0.0008640289306640625,
  0.0009436607360839844,
  0.000835418701171875,
  0.0008630752563476562)}

## NN

In [45]:
xTrain = pd.DataFrame({'Movie_Id': df['Movie_Id'], 'Cust_Id': df['Cust_Id']})
yTrain = pd.DataFrame({'Rating': df['Rating']})

print(xTrain.head())
print(yTrain.head())

   Movie_Id  Cust_Id
1         1  1488844
2         1   822109
3         1   885013
4         1    30878
5         1   823519
   Rating
1     3.0
2     5.0
3     4.0
4     4.0
5     3.0


In [47]:
# Input layer
ratings_input = Input(shape=(None,2))

# Embedding layers
dense0 = Dense(16, activation='relu')(ratings_input)

# Dense layers
#dense1 = Dense(128, activation='relu')(dense0)
dense2 = Dense(32, activation='relu')(dense0)

# Output layer
output = Dense(6,activation='softmax')(dense2)

# Model
model = Model(inputs=ratings_input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [52]:
model.fit(xTrain,yTrain, epochs=10, batch_size=64)

Epoch 1/10


16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193
Epoch 9/10
16/16 [==============================] - 0s 1ms/step - loss: 13.7193
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 13.7193


In [35]:
y_pred = model.predict(xTrain)

32/32 [==============================] - 0s 1ms/step


In [40]:
print(y_pred)
y_pred_custom = []
for i in y_pred:
    val = -1
    index = -1
    for j in range(len(i)):
        if i[j] > val:
            val = i[j]
            index = j
    y_pred_custom.append(index)
y_pred_custom = pd.DataFrame(y_pred_custom)
print(y_pred_custom)
print(yTrain)


[[0.         0.         0.         0.         0.         0.99999994]
 [0.         0.         0.         0.         0.         0.99999994]
 [0.         0.         0.         0.         0.         0.99999994]
 ...
 [0.         0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         0.         1.        ]]
     0
0    5
1    5
2    5
3    5
4    5
..  ..
992  5
993  5
994  5
995  5
996  5

[997 rows x 1 columns]
     Rating
1       3.0
2       5.0
3       4.0
4       4.0
5       3.0
..      ...
995     3.0
996     4.0
997     4.0
998     3.0
999     4.0

[997 rows x 1 columns]


In [39]:
mae = np.mean(y_pred_custom - yTrain)
print(f'MAE: {mae}')

MAE: 0        NaN
Rating   NaN
dtype: float64


/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
